In [120]:
from dotenv import load_dotenv
load_dotenv()

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from pprint import pprint
from dataclasses import dataclass
from typing import *
from pprint import pprint
import jsons

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

df = pd.read_csv('../resources/bubbleflexe-rv.csv', usecols=[1, 2], names=['bsides', 'tt'])
df = df.drop(labels=0, axis=0).reset_index(drop=True)
for c in df.columns:
    df[c] = df[c].apply(lambda x: str(x).split(';'))
df['unified'] = df['bsides'] + df['tt']

In [12]:
unique_bsides = set()
for i in df['bsides']:
    unique_bsides.update(i)

unique_tt = set()
for i in df['tt']:
    unique_tt.update(i)

unique_songs = unique_bsides.union(unique_tt)
unique_songs.remove('nan')

## Questions
1. For a given song, what is the frequency list of all other songs to it? 
2. How to use the size of favorite songs?
3. What about normalizing for song popularity

In [156]:
class Song:
    # todo make init based on name and artist make a spotify req to populate the object's name and artist
    def __init__(self, name: str, artist: str = 'Red Velvet', id: str = None, spotify_af: dict = None, tags: list = None):
        self.name = name
        self.artist = artist
        self.id = id or self.__get_track_id()
        self.spotify_af = spotify_af or self.__get_audio_analysis()
        self.tags = tags
        # self.freq_list = self.__get_freq_list()
        # self.total_listens = self.freq_list[self.name]
        # self.percent_list = None # after all songs are created, then we can populate the "chance" list 
    
    def __get_track_id(self):
        r = spotify.search(q=f'{self.name} artist:{self.artist}', type='track')
        return r['tracks']['items'][0]['id']
    
    def __get_audio_analysis(self):
        ft = spotify.audio_features(self.id)[0]
        delkeys = ['type', 'id', 'uri', 'track_href', 'analysis_url']
        [ft.pop(x) for x in delkeys]
        return ft

    def __repr__(self):
        # TODO see if this is safe
        return f'{self.name} - {self.artist} - {str(self.tags)}'

    def __str__(self):
        return f'{self.name} - {self.artist} - {str(self.tags)}'

    def __key(self):
        return (self.name) # TODO THIS IS ONLY DUE TO WORKING WITH RV CONTENT.
        # TODO find out a better way to make SongCollection like a dictionary but with regex matching for song name; make easier to extract from responses

    def __hash__(self):
        # note that we are heavily breaking convention here
        return hash(self.__key())

    def __eq__(self, o):
        if isinstance(o, Song):
            # if self.name == o.name and self.artist == o.artist:
            #     return True
            return self.__key() == o.__key()
        elif isinstance(o, str):
            # this is more useful for when there are multiple artists. we are only working with RV songs, so it's looser
            # if o == str(self)[:str(self).rfind('-')-1]:
            #     return True
            if o.lower() == self.name.lower():
                return True
        return False

class SongCollection:
    def __init__(self, songs: list, responses: pd.DataFrame):
        self.songs = songs # TODO create songs from responses
        self.responses = responses

    def get(self, match):
        return self.__get_by_eq(match)

    def __get_by_name(self, name):
        for s in self.songs:
            if name == s.name:
                return s
        raise Exception('no matching Song found')

    def __get_by_eq(self, obj):
        r = list(filter(lambda s: s == obj, self.songs))
        if len(r) > 1:
            raise Exception('too many songs matched')
        elif len(r) == 0:
            raise Exception('no matching Song found')
        return r[0]

    def get_song_names(self):
        return [s.name for s in self.songs]

    def get_freq_list(self):
        flist = {}
        for s in self.songs:
            flist[s] = 0
        for u in self.responses['unified']:
            for s in u:
                if s == 'nan':
                    continue
                flist[s] += 1
        return {k: v for k, v in sorted(flist.items(), key=lambda item: item[1])}

    def get_song_freq_list(self, songmatch):
        song = self.get(songmatch)

        flist = {}
        for s in self.songs:
            flist[s] = 0
        mask = self.responses['unified'].apply(lambda l: song.name in l)
        entries = self.responses[mask]['unified']
        for u in entries:
            for s in u:
                if s == 'nan':
                    continue
                flist[s] += 1

        return {k: v for k, v in sorted(flist.items(), key=lambda item: item[1])}

    def get_song_percent_list(self, songmatch):
        flist = self.get_song_freq_list(songmatch)
        root_flist = self.get_freq_list()
        for s in flist:
            flist[s] /= root_flist[s]
            flist[s] = round(flist[s], 2)
        return {k: v for k, v in sorted(flist.items(), key=lambda item: item[1])}

    def update():
        pass
    def add():
        pass
    def pop():
        pass
    
    # @staticmethod
    # def create_songs(names, artist):
    #     # move to song collection
    #     songdict = {}
    #     for s in names:
    #         songdict[s] = Song(s, artist)
    #     for s in songdict.values():
    #         s.construct_percent_list()
    #     # for s in songs
        
    
    



In [134]:
songs = [Song(name) for name in unique_songs]

In [157]:
col = SongCollection(responses=df, songs=songs)

In [158]:
col.get_song_percent_list('LP')

{Psycho - Red Velvet - None: 0.38,
 Bad Boy - Red Velvet - None: 0.41,
 Peek-A-Boo - Red Velvet - None: 0.42,
 Monster - Red Velvet - None: 0.43,
 Kingdom Come - Red Velvet - None: 0.43,
 Red Flavor - Red Velvet - None: 0.44,
 You Better Know - Red Velvet - None: 0.44,
 Sunny Side Up! - Red Velvet - None: 0.45,
 Russian Roulette - Red Velvet - None: 0.46,
 In & Out - Red Velvet - None: 0.46,
 Ice Cream Cake - Red Velvet - None: 0.47,
 Dumb Dumb - Red Velvet - None: 0.47,
 Naughty - Red Velvet - None: 0.47,
 Automatic - Red Velvet - None: 0.48,
 I Just - Red Velvet - None: 0.48,
 Zimzalabim - Red Velvet - None: 0.48,
 RBB - Red Velvet - None: 0.48,
 Feel Good - Red Velvet - None: 0.49,
 Oh Boy - Red Velvet - None: 0.49,
 One of These Nights - Red Velvet - None: 0.49,
 Umpah Umpah - Red Velvet - None: 0.49,
 Cool Hot Sweet Love - Red Velvet - None: 0.5,
 Candy - Red Velvet - None: 0.5,
 Body Talk - Red Velvet - None: 0.5,
 Zoo - Red Velvet - None: 0.5,
 Butterflies - Red Velvet - None: 0

In [161]:
with open('songcol', 'w') as f:
    f.write(jsons.dumps(col))